# Topic Classifier Test Notebook

Bu notebook, önceden eğitilmiş topic classifier modellerini test etmek için kullanılır.

## Özellikler:
- **Dosyadan Test**: CSV dosyasından test verilerini yükleyebilirsiniz
- **Manuel Test**: Kendi yazdığınız metinleri test edebilirsiniz
- **Model Karşılaştırması**: Tüm eğitilmiş modelleri karşılaştırabilirsiniz
- **Detaylı Analiz**: Confusion matrix ve classification report

## Gereksinimler:
Eğitim notebook'unun başarıyla çalışmış olması ve aşağıdaki dosyaların `models/` klasöründe bulunması:
- `feature_vectorizer.pkl`
- `training_config.pkl`
- Model dosyaları (örn: `logisticregression_model.pkl`)
- `pca_transformer.pkl`

## 1. Kütüphane İmportları ve Konfigürasyon

Bu bölüm, training notebook'unda kullanılan tüm sınıfları ve kaydedilen model bileşenlerini yükler.

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import time
import joblib
import pickle
import warnings
import re
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from tqdm import tqdm

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.feature_extraction import _stop_words
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

warnings.filterwarnings("ignore")

print("Kütüphaneler başarıyla yüklendi!")
print(f"Çalışma dizini: {os.getcwd()}")
print(f"Joblib versiyonu: {joblib.__version__}")
print(f"Python versiyonu: {sys.version.split()[0]}")

models_dir = "models"
if os.path.exists(models_dir):
    print(f"Models klasörü bulundu")
    files = [f for f in os.listdir(models_dir) if f.endswith('.pkl')]
    print(f"Mevcut .pkl dosyaları: {len(files)} adet")
    for f in sorted(files):
        print(f"   • {f}")
else:
    print(f"Models klasörü bulunamadı!")
    print("Çözüm: Training notebook'unu çalıştırın!")

Kütüphaneler başarıyla yüklendi!
Çalışma dizini: c:\Users\alper\OneDrive\Masaüstü\dersler\yap470\topic_calssifier\2.veriseti
Joblib versiyonu: 1.5.1
Python versiyonu: 3.11.9
Models klasörü bulundu
Mevcut .pkl dosyaları: 11 adet
   • feature_vectorizer.pkl
   • gradientboosting_model.pkl
   • gradientboosting_results.pkl
   • logisticregression_model.pkl
   • logisticregression_results.pkl
   • mlp_model.pkl
   • mlp_results.pkl
   • pca_transformer.pkl
   • svm_model.pkl
   • svm_results.pkl
   • training_config.pkl


## 2. Training Sınıflarının Tanımlanması

**Önemli:** Pickle dosyalarını yükleyebilmek için training notebook'unda kullanılan sınıfların burada da tanımlanması gerekir. Bu sınıflar training'deki aynı yapıdadır.

## 2. Kaydedilen Bileşenleri Yükleme

In [2]:
# Gerekli sınıf tanımları (pickle yükleme için)
from dataclasses import dataclass
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.feature_extraction import _stop_words
import re
import joblib
import os
import pickle
import numpy as np

# ============================================================================
# TRAINING NOTEBOOK'UNDAKI SINIF TANIMLARI
# ============================================================================

@dataclass
class Config:
    """Training konfigürasyonu - Training notebook'undaki aynı yapı"""
    # Data paths
    train_path: str = "archive/train.csv"
    test_path: str = "archive/test.csv"
    glove_dir: str = "glove"
    models_dir: str = "models"
    cache_dir: str = "cache"
    
    # GloVe settings
    glove_dim: int = 100
    
    # TF-IDF settings
    tfidf_max_features: int = 3000
    
    # KNN settings for missing words
    knn_neighbors: int = 5
    
    # PCA settings
    pca_components: int = 100
    
    # Cross-validation
    cv_folds: int = 3
    
    # Random search
    random_search_iter: int = 20
    random_state: int = 42
    
    # Strategy settings
    use_predefined_params: bool = True
    use_hybrid_vectors: bool = True
    
    # Categories
    categories: List[int] = None
    
    def __post_init__(self):
        if self.categories is None:
            self.categories = list(range(14))  # 0-13 arası 14 kategori
        
        # Create directories
        os.makedirs(self.models_dir, exist_ok=True)
        os.makedirs(self.cache_dir, exist_ok=True)
    
    @property
    def hybrid_vector_dim(self) -> int:
        """Calculate hybrid vector dimension"""
        if self.use_hybrid_vectors:
            return self.glove_dim * len(self.categories)  # 100 * 14 = 1400D
        else:
            return self.glove_dim

class TextPreprocessor:
    """Text preprocessing utilities - Training'deki aynı yapı"""
    
    @staticmethod
    def clean_text(text: str) -> str:
        """Advanced text cleaning"""
        if not isinstance(text, str):
            return ""
        
        text = text.lower()
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\b\w{1,2}\b', '', text)
        text = re.sub(r'\b\w{15,}\b', '', text)
        text = re.sub(r'(.)\1{2,}', r'\1\1', text)
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text

class TFIDFBasedKNNHandler:
    """TF-IDF based KNN handler for missing words - Training'deki aynı yapı"""
    
    def __init__(self, embeddings_index: Dict, word_score_cache: Dict, config: Config):
        self.embeddings_index = embeddings_index
        self.word_score_cache = word_score_cache
        self.config = config

class FeatureVectorizer:
    """Main feature vectorization class - Training'deki aynı yapı"""
    
    def __init__(self, embeddings_index: Dict, word_score_cache: Dict, config: Config):
        self.embeddings_index = embeddings_index.copy()
        self.word_score_cache = word_score_cache
        self.config = config
        self.missing_word_handler = TFIDFBasedKNNHandler(embeddings_index, word_score_cache, config)
        self.missing_word_cache = {}
        self.pca = PCA(n_components=config.pca_components, random_state=config.random_state)
        self.is_fitted = False
    
    def text_to_vector(self, text: str, target_category: int) -> np.ndarray:
        """Convert single text to vector"""
        cleaned_text = TextPreprocessor.clean_text(text)
        tokens = cleaned_text.split()
        stop_words = _stop_words.ENGLISH_STOP_WORDS
        filtered_tokens = [t for t in tokens if t not in stop_words and len(t) >= 3]
        
        if not filtered_tokens:
            if self.config.use_hybrid_vectors:
                return np.zeros(self.config.hybrid_vector_dim)
            else:
                return np.zeros(self.config.glove_dim)
        
        weighted_vectors = []
        total_weight = 0
        
        for token in filtered_tokens:
            if token in self.embeddings_index:
                base_vector = self.embeddings_index[token]
                weight = self._calculate_word_weight(token, target_category)
                
                if self.config.use_hybrid_vectors:
                    hybrid_vector = self._create_hybrid_vector(base_vector, token)
                    weighted_vectors.append(hybrid_vector * weight)
                else:
                    weighted_vectors.append(base_vector * weight)
                
                total_weight += weight
            else:
                # Missing word handling
                if token in self.missing_word_cache:
                    base_vector = self.missing_word_cache[token]
                else:
                    base_vector = np.random.normal(0, 0.1, size=self.config.glove_dim)
                    self.missing_word_cache[token] = base_vector
                    self.embeddings_index[token] = base_vector
                
                if self.config.use_hybrid_vectors:
                    hybrid_vector = self._create_hybrid_vector(base_vector, token)
                    weighted_vectors.append(hybrid_vector * 0.2)
                else:
                    weighted_vectors.append(base_vector * 0.2)
                
                total_weight += 0.2
        
        if not weighted_vectors or total_weight == 0:
            if self.config.use_hybrid_vectors:
                return np.zeros(self.config.hybrid_vector_dim)
            else:
                return np.zeros(self.config.glove_dim)
        
        return np.sum(weighted_vectors, axis=0) / total_weight
    
    def _create_hybrid_vector(self, base_vector: np.ndarray, word: str) -> np.ndarray:
        """Create hybrid vector: 100D GloVe → 1400D (14×TF-IDF weighted)"""
        hybrid_parts = []
        
        for category in self.config.categories:
            tfidf_score = self.word_score_cache.get(word, {}).get(category, 0.0)
            category_weighted_vector = base_vector * tfidf_score
            hybrid_parts.append(category_weighted_vector)
        
        return np.concatenate(hybrid_parts)
    
    def _calculate_word_weight(self, word: str, target_category: int) -> float:
        """Calculate word weight based on TF-IDF scores"""
        if word not in self.word_score_cache:
            return 0.1
        
        category_scores = [
            self.word_score_cache[word].get(cat, 0.0) 
            for cat in self.config.categories
        ]
        
        target_score = category_scores[target_category]  # 0-indexed kategori
        
        if target_score <= 0:
            return 0.1
        
        other_scores = [
            category_scores[i] for i in range(len(self.config.categories))
            if i != target_category
        ]
        
        mean_other = np.mean(other_scores) if other_scores else 0
        std_other = np.std(other_scores) if len(other_scores) > 1 else 0.01
        
        if std_other > 0:
            z_score = (target_score - mean_other) / std_other
            distinctiveness = max(0, min(2, z_score)) / 2.0
        else:
            distinctiveness = min(target_score / (mean_other + 0.01), 2.0) / 2.0
        
        final_weight = target_score * (1.0 + distinctiveness)
        return min(final_weight, 2.0)
    
    def transform_pca(self, vectors: np.ndarray) -> np.ndarray:
        """Transform vectors using fitted PCA"""
        if not self.is_fitted:
            raise ValueError("PCA not fitted yet!")
        return self.pca.transform(vectors)

# ============================================================================
# MODEL YÜKLEME VE TEST SINIFI
# ============================================================================

class ModelLoader:
    """Kaydedilen modelleri ve bileşenleri yüklemek için sınıf"""
    
    def __init__(self, models_dir: str = "models"):
        self.models_dir = models_dir
        self.vectorizer = None
        self.config = None
        self.models = {}
        self.category_names = {
            0: 'Company', 1: 'Sports', 2: 'Business', 3: 'Science/Tech', 
            4: 'Politics', 5: 'Education', 6: 'Health', 7: 'Geography',
            8: 'Art/Media', 9: 'Nature/Biology', 10: 'Biology', 11: 'Music',
            12: 'Film', 13: 'Literature'
        }
    
    def _safe_load_pickle(self, file_path: str):
        """Güvenli pickle yükleme - hem joblib hem de pickle dener"""
        try:
            # Önce joblib ile dene (training'de bu kullanıldı)
            obj = joblib.load(file_path)
            print(f"   Joblib ile yüklendi")
            return obj
        except Exception as e:
            print(f"   Joblib hatası: {e}")
            try:
                # Pickle ile dene
                with open(file_path, 'rb') as f:
                    obj = pickle.load(f)
                print(f"   Pickle ile yüklendi")
                return obj
            except Exception as e2:
                print(f"   Pickle hatası: {e2}")
                raise Exception(f"Dosya yüklenemedi (joblib: {e}, pickle: {e2})")
    
    def load_essential_components(self):
        """Temel bileşenleri yükle"""
        print("\nTemel bileşenler yükleniyor...")
        
        # Models klasörünü kontrol et
        if not os.path.exists(self.models_dir):
            raise FileNotFoundError(f"Models klasörü bulunamadı: {self.models_dir}")
        
        print(f"Models klasörü içeriği: {os.listdir(self.models_dir)}")
        
        # Config'i yükle
        config_path = os.path.join(self.models_dir, "training_config.pkl")
        if os.path.exists(config_path):
            try:
                print(f"Config yükleniyor...")
                self.config = self._safe_load_pickle(config_path)
                print(f"   Hybrid vectors: {self.config.use_hybrid_vectors}")
                print(f"   Vector dim: {self.config.hybrid_vector_dim}D")
                print(f"   PCA components: {self.config.pca_components}")
            except Exception as e:
                print(f"Config yükleme hatası: {e}")
                raise Exception(f"Training config yüklenemedi: {e}")
        else:
            raise FileNotFoundError(f"Training config bulunamadı: {config_path}")
        
        # FeatureVectorizer'ı yükle
        vectorizer_path = os.path.join(self.models_dir, "feature_vectorizer.pkl")
        if os.path.exists(vectorizer_path):
            try:
                print(f"FeatureVectorizer yükleniyor...")
                self.vectorizer = self._safe_load_pickle(vectorizer_path)
                print(f"   PCA fitted: {self.vectorizer.is_fitted}")
                
            except Exception as e:
                print(f"FeatureVectorizer yükleme hatası: {e}")
                raise Exception(f"FeatureVectorizer yüklenemedi: {e}")
        else:
            raise FileNotFoundError(f"FeatureVectorizer bulunamadı: {vectorizer_path}")
    
    def load_available_models(self):
        """Mevcut modelleri yükle"""
        print(f"\nModeller yükleniyor...")
        
        model_files = {
            'LogisticRegression': 'logisticregression_model.pkl',
            'SVM': 'svm_model.pkl', 
            'MLP': 'mlp_model.pkl',
            'GradientBoosting': 'gradientboosting_model.pkl'
        }
        
        loaded_count = 0
        for model_name, filename in model_files.items():
            model_path = os.path.join(self.models_dir, filename)
            if os.path.exists(model_path):
                try:
                    print(f"   {model_name} yükleniyor...")
                    model = self._safe_load_pickle(model_path)
                    self.models[model_name] = model
                    loaded_count += 1
                except Exception as e:
                    print(f"   {model_name} yüklenemedi: {e}")
            else:
                print(f"   {model_name} bulunamadı")
        
        if loaded_count == 0:
            raise FileNotFoundError("Hiçbir model dosyası bulunamadı!")
        
        print(f"\nToplam {loaded_count} model yüklendi: {list(self.models.keys())}")
    
    def predict_single_text(self, text: str, target_category: int, model_name: str = None) -> Dict:
        """Tek metin için prediction yap"""
        if not self.vectorizer or not self.models:
            raise ValueError("Bileşenler yüklenmemiş!")
        
        # Vektör oluştur
        vector = self.vectorizer.text_to_vector(text, target_category)
        
        results = {}
        models_to_test = [model_name] if model_name and model_name in self.models else self.models.keys()
        
        for name in models_to_test:
            try:
                model = self.models[name]
                
                if name == 'LogisticRegression':
                    # LogisticRegression full-dimensional (1400D) vektör kullanır
                    vector_input = vector.reshape(1, -1)
                else:
                    # Diğer modeller PCA'lı (100D) vektör kullanır
                    vector_input = self.vectorizer.transform_pca(vector.reshape(1, -1))
                
                prediction = model.predict(vector_input)[0]
                results[name] = {
                    'prediction': prediction,
                    'category_name': self.category_names[prediction]
                }
            except Exception as e:
                results[name] = {'error': str(e)}
        
        return results
    
    def predict_batch(self, texts: List[str], target_categories: List[int], model_name: str = None) -> Dict:
        """Toplu prediction"""
        if not self.vectorizer or not self.models:
            raise ValueError("Bileşenler yüklenmemiş!")
        
        # Vektörleri oluştur
        vectors = []
        for text, target_cat in zip(texts, target_categories):
            vector = self.vectorizer.text_to_vector(text, target_cat)
            vectors.append(vector)
        
        vectors_array = np.array(vectors)  # Full-dimensional (1400D)
        
        # PCA'lı vektörleri hesapla (sadece gerekli modeller varsa)
        models_to_process = [model_name] if model_name and model_name in self.models else self.models.keys()
        needs_pca = any(name != 'LogisticRegression' for name in models_to_process)
        
        if needs_pca:
            vectors_pca = self.vectorizer.transform_pca(vectors_array)
        
        # Predictions
        results = {}
        for name in models_to_process:
            try:
                model = self.models[name]
                
                if name == 'LogisticRegression':
                    predictions = model.predict(vectors_array)
                else:
                    predictions = model.predict(vectors_pca)
                
                results[name] = predictions
                
            except Exception as e:
                results[name] = {'error': str(e)}
        
        return results

print("Tüm sınıflar tanımlandı!")
print("Training notebook'undaki aynı yapılar kullanılıyor")
print("Pickle dosyaları yüklenmeye hazır")

Tüm sınıflar tanımlandı!
Training notebook'undaki aynı yapılar kullanılıyor
Pickle dosyaları yüklenmeye hazır


## 3. Bileşenleri Yükle ve CSV Test Verileri Hazırla

In [3]:
# ============================================================================
# KAYDEDILEN BİLEŞENLERİ YÜKLE
# ============================================================================

print("Sistem kontrolleri...")
print(f"Çalışma dizini: {os.getcwd()}")
print(f"Python versiyonu: {sys.version.split()[0]}")

# Models klasörünü kontrol et
models_dir = "models"
if not os.path.exists(models_dir):
    print(f"Models klasörü bulunamadı!")
    print("Çözüm: Training notebook'unu çalıştırın!")
    sys.exit()

print(f"Models klasörü bulundu")
pkl_files = [f for f in os.listdir(models_dir) if f.endswith('.pkl')]
print(f"Mevcut .pkl dosyaları ({len(pkl_files)} adet):")
for f in sorted(pkl_files):
    size_mb = os.path.getsize(os.path.join(models_dir, f)) / (1024*1024)
    print(f"   • {f} ({size_mb:.1f} MB)")

# ModelLoader'ı başlat ve bileşenleri yükle
print(f"\n" + "="*60)
print("BILEŞEN YÜKLEME")
print("="*60)

try:
    loader = ModelLoader(models_dir)
    
    # Temel bileşenleri yükle
    loader.load_essential_components()
    
    # Modelleri yükle  
    loader.load_available_models()
    
    print(f"\nTÜM BİLEŞENLER BAŞARIYLA YÜKLENDİ!")
    print(f"Vectorizer türü: {'Hybrid' if loader.config.use_hybrid_vectors else 'Standard'}")
    print(f"Vector boyutu: {loader.config.hybrid_vector_dim}D")
    print(f"PCA boyutu: {loader.config.pca_components}D")
    print(f"Yüklenen modeller: {list(loader.models.keys())}")
    
except Exception as e:
    print(f"\nYÜKLEME HATASI: {e}")
    print(f"Hata türü: {type(e).__name__}")
    
    # Detaylı hata analizi
    import traceback
    print(f"\nDetaylı hata raporu:")
    print(traceback.format_exc())
    
    print(f"\nÇÖZÜMLER:")
    print("1. Training notebook'unu (topic_classifier_training.ipynb) çalıştırın")
    print("2. Tüm hücreler başarıyla çalıştığından emin olun")
    print("3. Models klasöründe gerekli .pkl dosyalarının olduğunu kontrol edin")
    print("4. Python sürüm uyumluluğunu kontrol edin")
    sys.exit()

# ============================================================================
# CSV TEST VERİLERİNİ YÜKLE
# ============================================================================

print(f"\n" + "="*60)
print("CSV TEST VERİLERİ YÜKLEME")
print("="*60)

test_file_path = "archive/test.csv"

def load_test_data(file_path: str):
    """Test dosyasını yükle"""
    try:
        df = pd.read_csv(file_path, header=None)
        df.columns = ["label", "title", "description"]
        
        # Remove header rows that might contain text values
        df = df[df["label"] != "Class Index"]
        df = df[df["label"] != "label"]  # Remove column header if present
        
        # Remove rows where label is not numeric
        df = df[pd.to_numeric(df["label"], errors='coerce').notna()]
        df["label"] = df["label"].astype(int)
        df["text"] = df["title"] + " " + df["description"]
        
        texts = df["text"].tolist()
        labels = df["label"].tolist()
        
        print(f"Test dosyası yüklendi: {len(texts)} sample")
        print(f"Label dağılımı:")
        label_counts = df['label'].value_counts().sort_index()
        for label, count in label_counts.items():
            print(f"   {label} ({loader.category_names[label]}): {count} sample")
        
        return texts, labels, df
        
    except Exception as e:
        print(f"Test dosyası yüklenirken hata: {e}")
        return [], [], pd.DataFrame()

if os.path.exists(test_file_path):
    test_texts, test_labels, test_df = load_test_data(test_file_path)
    
    if test_texts:
        print(f"\nİlk 3 test örneği:")
        for i in range(min(3, len(test_texts))):
            print(f"{i+1}. [{loader.category_names[test_labels[i]]}] {test_texts[i][:80]}...")
            
        # Hızlı performans testi (ilk 50 sample)
        print(f"\nHızlı performans testi (ilk 50 sample)...")
        quick_texts = test_texts[:50]
        quick_labels = test_labels[:50]
        
        start_time = time.time()
        quick_results = loader.predict_batch(quick_texts, quick_labels)
        test_time = time.time() - start_time
        
        print(f"Hızlı test sonuçları ({len(quick_texts)} sample, {test_time:.2f}s):")
        for model_name, predictions in quick_results.items():
            if isinstance(predictions, np.ndarray):
                accuracy = accuracy_score(quick_labels, predictions)
                speed = len(quick_texts) / test_time
                print(f"   {model_name}: {accuracy:.3f} accuracy ({speed:.1f} samples/s)")
            else:
                print(f"   {model_name}: Hata - {predictions.get('error', 'Bilinmeyen')}")
        
        print(f"\nCSV test verileri hazır! Toplam: {len(test_texts)} sample")
        print(f"Test hızı: ~{len(quick_texts)/test_time:.0f} samples/s")
        
    else:
        print("Test verileri yüklenemedi!")
        test_texts, test_labels, test_df = [], [], pd.DataFrame()
else:
    print(f"Test dosyası bulunamadı: {test_file_path}")
    test_texts, test_labels, test_df = [], [], pd.DataFrame()

print(f"\nHAZIR! Şimdi manuel test veya detaylı analiz yapabilirsiniz.")

Sistem kontrolleri...
Çalışma dizini: c:\Users\alper\OneDrive\Masaüstü\dersler\yap470\topic_calssifier\2.veriseti
Python versiyonu: 3.11.9
Models klasörü bulundu
Mevcut .pkl dosyaları (11 adet):
   • feature_vectorizer.pkl (623.8 MB)
   • gradientboosting_model.pkl (0.7 MB)
   • gradientboosting_results.pkl (0.3 MB)
   • logisticregression_model.pkl (0.2 MB)
   • logisticregression_results.pkl (0.3 MB)
   • mlp_model.pkl (0.2 MB)
   • mlp_results.pkl (0.3 MB)
   • pca_transformer.pkl (1.1 MB)
   • svm_model.pkl (16.0 MB)
   • svm_results.pkl (0.3 MB)
   • training_config.pkl (0.0 MB)

BILEŞEN YÜKLEME

Temel bileşenler yükleniyor...
Models klasörü içeriği: ['feature_vectorizer.pkl', 'gradientboosting_model.pkl', 'gradientboosting_results.pkl', 'logisticregression_model.pkl', 'logisticregression_results.pkl', 'mlp_model.pkl', 'mlp_results.pkl', 'pca_transformer.pkl', 'svm_model.pkl', 'svm_results.pkl', 'training_config.pkl']
Config yükleniyor...
   Joblib ile yüklendi
   Hybrid vectors: 

## 4. Manuel Test - Kendi Metninizi Test Edin

Bu bölümde kendi yazdığınız metinleri test edebilirsiniz. Aşağıdaki hücreyi düzenleyerek istediğiniz metni ve beklenen kategoriyi belirleyebilirsiniz.

**Kategoriler:**
- 0: Company (Şirket/Organizasyon)
- 1: Sports (Spor)
- 2: Business (İş/Ekonomi)
- 3: Science/Tech (Bilim/Teknoloji)
- 4: Politics (Politika)
- 5: Education (Eğitim)
- 6: Health (Sağlık)
- 7: Geography (Coğrafya)
- 8: Art/Media (Sanat/Medya)
- 9: Nature/Biology (Doğa/Biyoloji)
- 10: Biology (Biyoloji detay)
- 11: Music (Müzik)
- 12: Film (Film)
- 13: Literature (Edebiyat)

In [4]:
# MANUEL TEST - KENDİ METNİNİZİ BURAYA YAZIN
print("="*60)
print("MANUEL METİN TESTİ")
print("="*60)

# BURASI SİZİN DÜZENLEYECEĞINIZ BÖLÜM
my_text = "Apple reported record quarterly earnings beating analyst expectations"
my_expected_category = 2  # 0: Company, 1: Sports, 2: Business, 3: Science/Tech, vs.

# Test örnekleri (isterseniz bunları da deneyebilirsiniz)
sample_tests = [
    ("Manchester United won the Champions League final against Real Madrid", 1),
    ("NASA successfully launched a new rover to Mars surface", 3), 
    ("European Union announces new trade sanctions against Russia", 4),
    ("Tesla stock price surged after strong delivery numbers", 2),
    ("Scientists discovered a new species of dinosaur in Argentina", 9),
    ("Harvard University announces new scholarship program", 5),
    ("WHO reports new breakthrough in cancer treatment", 6),
    ("Amazon rainforest deforestation reaches record levels", 9),
    ("Netflix releases new documentary about climate change", 12),
    ("Beethoven's lost symphony discovered in German archive", 11)
]

print(f"Test Metni: {my_text}")
print(f"Beklenen Kategori: {loader.category_names[my_expected_category]}")
print("\n" + "-"*50)

if my_text and my_text.strip():
    # Ana test
    results = loader.predict_single_text(my_text, my_expected_category)
    
    print("MODEL TAHMİNLERİ:")
    correct_predictions = 0
    total_models = len([r for r in results.values() if 'error' not in r])
    
    for model_name, result in results.items():
        if 'error' not in result:
            prediction = result['prediction']
            predicted_category = result['category_name']
            is_correct = prediction == my_expected_category
            
            if is_correct:
                correct_predictions += 1
                status = "DOĞRU"
            else:
                status = "YANLIŞ"
            
            print(f"   {model_name}: {predicted_category} ({status})")
        else:
            print(f"   {model_name}: HATA - {result['error']}")
    
    if total_models > 0:
        consensus_rate = correct_predictions / total_models
        print(f"\nModel Consensus: {correct_predictions}/{total_models} ({consensus_rate:.1%})")
        
        if consensus_rate >= 0.75:
            print("Güçlü consensus - Modeller genel olarak aynı fikirde!")
        elif consensus_rate >= 0.5:
            print("Orta consensus - Modeller kısmen hemfikir")
        else:
            print("Zayıf consensus - Modeller farklı tahminlerde bulunuyor")

    # Örnek testleri de göster
    print(f"\n" + "="*40)
    print("ÖRNEK TESTLER (İsteğe bağlı)")
    print("="*40)
    
    for i, (sample_text, expected_cat) in enumerate(sample_tests[:3], 1):
        print(f"\n{i}. [{loader.category_names[expected_cat]}] {sample_text[:60]}...")
        sample_results = loader.predict_single_text(sample_text, expected_cat)
        
        print("   Model tahminleri:")
        for model_name, result in sample_results.items():
            if 'error' not in result:
                predicted_category = result['category_name']
                print(f"     {model_name}: {predicted_category}")
            else:
                print(f"     {model_name}: HATA")

else:
    print("Lütfen 'my_text' değişkenini yukarıda düzenleyin!")
    print("\nÖrnek kullanım:")
    print('my_text = "Your text here..."')
    print('my_expected_category = 1  # Sports kategori için')

MANUEL METİN TESTİ
Test Metni: Apple reported record quarterly earnings beating analyst expectations
Beklenen Kategori: Business

--------------------------------------------------
MODEL TAHMİNLERİ:
   LogisticRegression: Literature (YANLIŞ)
   SVM: Company (YANLIŞ)
   MLP: Company (YANLIŞ)
   GradientBoosting: Literature (YANLIŞ)

Model Consensus: 0/4 (0.0%)
Zayıf consensus - Modeller farklı tahminlerde bulunuyor

ÖRNEK TESTLER (İsteğe bağlı)

1. [Sports] Manchester United won the Champions League final against Rea...
   Model tahminleri:
     LogisticRegression: Nature/Biology
     SVM: Company
     MLP: Company
     GradientBoosting: Education

2. [Science/Tech] NASA successfully launched a new rover to Mars surface...
   Model tahminleri:
     LogisticRegression: Education
     SVM: Education
     MLP: Education
     GradientBoosting: Education

3. [Politics] European Union announces new trade sanctions against Russia...
   Model tahminleri:
     LogisticRegression: Company
     SV

## 5. Detaylı Analiz - Comprehensive Model Evaluation

Bu bölümde tüm modellerin performansını detaylı şekilde analiz ediyoruz:

### Analizler:
- **Confusion Matrix**: Her model için karışıklık matrisi
- **Kategori Bazında Doğruluk**: Hangi kategoride hangi model en başarılı
- **Model Hızları**: Prediction süreleri karşılaştırması  
- **En İyi Model Seçimi**: Farklı kriterler için öneriler
- **Detaylı Performans Metrikleri**: Precision, Recall, F1-Score

In [5]:
# DETAYLI ANALİZ VE MODEL KARŞILAŞTIRMASI
# Clear previous outputs to prevent duplication
import sys
from IPython.display import clear_output

# Tekrar çalıştırma kontrolü
if 'analysis_running' in globals() and analysis_running:
    print("Analiz zaten çalışıyor! Lütfen bekleyin...")
else:
    analysis_running = True
    
    print("="*80)
    print("COMPREHENSIVE MODEL EVALUATION & ANALYSIS")
    print("="*80)

    # Önceki analiz değişkenlerini temizle
    if 'all_results' in locals():
        del all_results
    if 'timing_results' in locals():
        del timing_results

    if not test_texts or not test_labels:
        print("Test verileri bulunamadı! Önce CSV verilerini yükleyin.")
        analysis_running = False
    else:
        analysis_size = min(70000, len(test_texts))
        analysis_texts = test_texts[:analysis_size]
        analysis_labels = test_labels[:analysis_size]
        
        print(f"Analiz kapsamı: {analysis_size} test sample")
        print(f"Test edilen modeller: {list(loader.models.keys())}")
        
        # Test verisindeki kategorileri analiz et
        unique_labels = sorted(list(set(analysis_labels)))
        print(f"Test verisindeki kategoriler: {unique_labels}")
        
        # Tüm modeller için batch prediction ve performans ölçümü
        all_results = {}
        timing_results = {}
        
        print(f"\nTüm modellerle test yapılıyor...")
        
        # Model listesini bir kez al ve tekrarlama engelle
        model_list = list(loader.models.keys())
        processed_models = set()
        
        for model_name in model_list:
            # Tekrarlama kontrolü
            if model_name in processed_models:
                continue
            processed_models.add(model_name)
            
            try:
                start_time = time.time()
                predictions = loader.predict_batch(analysis_texts, analysis_labels, model_name)[model_name]
                end_time = time.time()
                
                if isinstance(predictions, np.ndarray):
                    all_results[model_name] = predictions
                    timing_results[model_name] = end_time - start_time
                    print(f"   {model_name}: {len(predictions)} prediction in {end_time - start_time:.3f}s")
                else:
                    print(f"   {model_name}: Error - {predictions.get('error', 'Unknown')}")
                    
            except Exception as e:
                print(f"   {model_name}: Exception - {str(e)}")
        
        # all_results dict'inin boş olup olmadığını kontrol et
        if all_results:
            # 1. GENEL PERFORMANS ÖZETİ
            print(f"\n" + "="*60)
            print("1. GENEL PERFORMANS ÖZETİ")
            print("="*60)
            
            performance_data = []
            
            for model_name, predictions in all_results.items():
                accuracy = accuracy_score(analysis_labels, predictions)
                speed = len(predictions) / timing_results[model_name]
                
                performance_data.append({
                    'Model': model_name,
                    'Accuracy': f"{accuracy:.4f}",
                    'Speed (samples/s)': f"{speed:.1f}",
                    'Time (s)': f"{timing_results[model_name]:.3f}",
                    'Correct': sum(predictions == np.array(analysis_labels)),
                    'Total': len(analysis_labels)
                })
            
            performance_df = pd.DataFrame(performance_data)
            print(performance_df.to_string(index=False))
            
            # En iyi modelleri belirle
            best_accuracy = max(performance_data, key=lambda x: float(x['Accuracy']))
            fastest_model = max(performance_data, key=lambda x: float(x['Speed (samples/s)']))
            
            print(f"\nEn yüksek accuracy: {best_accuracy['Model']} ({best_accuracy['Accuracy']})")
            print(f"En hızlı model: {fastest_model['Model']} ({fastest_model['Speed (samples/s)']} samples/s)")
            
            # 2. CONFUSION MATRIX ANALİZİ
            print(f"\n" + "="*60)
            print("2. CONFUSION MATRIX ANALİZİ")
            print("="*60)
            
            best_model_name = best_accuracy['Model']
            best_predictions = all_results[best_model_name]
            
            print(f"\nEn iyi model ({best_model_name}) için Confusion Matrix:")
            cm = confusion_matrix(analysis_labels, best_predictions)
            
            # Sadece test verisinde bulunan kategorileri kullan
            all_category_names = ['Company', 'Sports', 'Business', 'Science/Tech', 'Politics', 
                                'Education', 'Health', 'Geography', 'Art/Media', 'Nature/Biology', 
                                'Biology', 'Music', 'Film', 'Literature']
            
            # Test verisindeki kategorilerin isimlerini al
            present_category_names = [all_category_names[i] for i in unique_labels]
            
            # Confusion matrix DataFrame'ini oluştur - boyutlar uyumlu olmalı
            print(f"Confusion Matrix boyutu: {cm.shape}")
            print(f"Kategori sayısı: {len(present_category_names)}")
            
            if cm.shape[0] == len(present_category_names):
                cm_df = pd.DataFrame(cm, index=present_category_names, columns=present_category_names)
                print(cm_df)
            else:
                print("Confusion Matrix:")
                print(cm)
                print(f"Satırlar: {present_category_names}")
                print(f"Sütunlar: {present_category_names}")
                # Manuel olarak yazdır
                print("\nConfusion Matrix (manuel):")
                for i, true_cat in enumerate(present_category_names):
                    row_str = f"{true_cat:12} |"
                    for j in range(cm.shape[1]):
                        row_str += f"{cm[i,j]:6d}"
                    print(row_str)
                print("             " + "".join([f"{cat[:6]:>6}" for cat in present_category_names]))
            
            # Confusion matrix analizini genişlet
            print(f"\nConfusion Matrix Analizi:")
            for i, category_idx in enumerate(unique_labels):
                category_name = all_category_names[category_idx]
                true_positives = cm[i, i]
                total_actual = cm[i, :].sum()
                total_predicted = cm[:, i].sum()
                
                if total_actual > 0:
                    recall = true_positives / total_actual
                    precision = true_positives / total_predicted if total_predicted > 0 else 0
                    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
                    
                    print(f"   {category_name}: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}")
            
            # 3. KATEGORİ BAZINDA PERFORMANS
            print(f"\n" + "="*60)
            print("3. KATEGORİ BAZINDA PERFORMANS")
            print("="*60)
            
            category_performance = {}
            
            for category_idx in unique_labels:
                category_name = all_category_names[category_idx]
                category_mask = np.array(analysis_labels) == category_idx
                category_true = np.array(analysis_labels)[category_mask]
                
                if sum(category_mask) > 0:  # Sadece var olan kategoriler için
                    print(f"\n{category_name} Kategorisi ({sum(category_mask)} sample):")
                    
                    category_accuracies = []
                    
                    for model_name, predictions in all_results.items():
                        category_pred = predictions[category_mask]
                        if len(category_pred) > 0:
                            category_acc = accuracy_score(category_true, category_pred)
                            category_accuracies.append((model_name, category_acc))
                            
                    # En iyi modeli bul
                    if category_accuracies:
                        category_accuracies.sort(key=lambda x: x[1], reverse=True)
                        best_for_category = category_accuracies[0]
                        
                        print(f"   En iyi model: {best_for_category[0]} ({best_for_category[1]:.3f})")
                        print("   Tüm modeller:")
                        for model, acc in category_accuracies:
                            print(f"     {model}: {acc:.3f}")
                        
                        category_performance[category_name] = category_accuracies
            
            # 4. MODEL KARŞILAŞTIRMA TABLOSU
            print(f"\n" + "="*60)
            print("4. MODEL KARŞILAŞTIRMA TABLOSU")
            print("="*60)
            
            comparison_table = []
            
            for model_name in all_results.keys():
                row = {'Model': model_name}
                
                # Genel accuracy
                overall_acc = accuracy_score(analysis_labels, all_results[model_name])
                row['Overall'] = f"{overall_acc:.3f}"
                
                # Kategori bazında accuracy (sadece mevcut kategoriler için)
                for category_idx in unique_labels:
                    category_name = all_category_names[category_idx]
                    category_mask = np.array(analysis_labels) == category_idx
                    if sum(category_mask) > 0:
                        category_true = np.array(analysis_labels)[category_mask]
                        category_pred = all_results[model_name][category_mask]
                        category_acc = accuracy_score(category_true, category_pred)
                        row[category_name] = f"{category_acc:.3f}"
                    else:
                        row[category_name] = "N/A"
                
                # Hız
                row['Speed'] = f"{len(all_results[model_name]) / timing_results[model_name]:.1f}"
                
                comparison_table.append(row)
            
            comparison_df = pd.DataFrame(comparison_table)
            print(comparison_df.to_string(index=False))
            
            # 5. YANLIŞ TAHMİN ANALİZİ
            print(f"\n" + "="*60)
            print("5. YANLIŞ TAHMİN ANALİZİ")
            print("="*60)
            
            # En iyi modelin yanlış tahminlerini analiz et
            wrong_predictions = []
            correct_count = 0
            
            for i, (true_label, pred_label) in enumerate(zip(analysis_labels, best_predictions)):
                if true_label != pred_label:
                    wrong_predictions.append({
                        'index': i,
                        'text': analysis_texts[i][:100] + "...",
                        'true': loader.category_names[true_label],
                        'predicted': loader.category_names[pred_label],
                        'true_idx': true_label,
                        'pred_idx': pred_label
                    })
                else:
                    correct_count += 1
            
            print(f"Doğru tahminler: {correct_count}")
            print(f"Yanlış tahminler: {len(wrong_predictions)}")
            print(f"Genel accuracy: {correct_count / (correct_count + len(wrong_predictions)):.3f}")
            
            # En çok karışan kategorileri bul
            error_patterns = {}
            for wp in wrong_predictions:
                pattern = f"{wp['true']} → {wp['predicted']}"
                error_patterns[pattern] = error_patterns.get(pattern, 0) + 1
            
            if error_patterns:
                print(f"\nEn sık karışan kategori çiftleri:")
                sorted_errors = sorted(error_patterns.items(), key=lambda x: x[1], reverse=True)
                for pattern, count in sorted_errors[:5]:
                    print(f"   {pattern}: {count} kez")
            
            # Örnek yanlış tahminler
            if wrong_predictions:
                print(f"\nÖrnek yanlış tahminler (ilk 5):")
                for i, wp in enumerate(wrong_predictions[:5], 1):
                    print(f"\n{i}. {wp['text']}")
                    print(f"   Gerçek: {wp['true']} | Tahmin: {wp['predicted']}")
            
            # 6. SONUÇ VE ÖNERİLER
            print(f"\n" + "="*60)
            print("6. SONUÇ VE ÖNERİLER")
            print("="*60)
            
            print(f"Test sonucu özeti:")
            print(f"   • Test edilen sample sayısı: {analysis_size}")
            print(f"   • Test verisindeki kategoriler: {len(unique_labels)} adet")
            print(f"   • En iyi genel performans: {best_accuracy['Model']} ({best_accuracy['Accuracy']})")
            print(f"   • En hızlı model: {fastest_model['Model']} ({fastest_model['Speed (samples/s)']} samples/s)")
            
            print(f"\nKullanım önerileri:")
            print(f"   Accuracy öncelikli: {best_accuracy['Model']}")
            print(f"   Hız öncelikli: {fastest_model['Model']}")
            
            # Dengeli öneri
            balanced_scores = []
            for perf in performance_data:
                # Normalize edilmiş accuracy ve speed skorlarını birleştir
                acc_score = float(perf['Accuracy'])
                speed_score = min(float(perf['Speed (samples/s)']) / 100, 1.0)  # Normalize
                balanced_score = (acc_score * 0.7) + (speed_score * 0.3)  # %70 accuracy, %30 speed
                balanced_scores.append((perf['Model'], balanced_score))
            
            balanced_best = max(balanced_scores, key=lambda x: x[1])
            print(f"   Dengeli seçim: {balanced_best[0]} (accuracy + speed optimized)")
            
            print(f"\nAnaliz tamamlandı! {time.strftime('%Y-%m-%d %H:%M:%S')}")
            
        else:
            print("Hiçbir model çalışmadı! Model yükleme sorunlarını kontrol edin.")
    
    # Reset flag
    analysis_running = False

COMPREHENSIVE MODEL EVALUATION & ANALYSIS
Analiz kapsamı: 70000 test sample
Test edilen modeller: ['LogisticRegression', 'SVM', 'MLP', 'GradientBoosting']
Test verisindeki kategoriler: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

Tüm modellerle test yapılıyor...
   LogisticRegression: 70000 prediction in 75.848s
   LogisticRegression: 70000 prediction in 75.848s
   SVM: 70000 prediction in 109.224s
   SVM: 70000 prediction in 109.224s
   MLP: 70000 prediction in 80.887s
   MLP: 70000 prediction in 80.887s
   GradientBoosting: 70000 prediction in 79.323s

1. GENEL PERFORMANS ÖZETİ
             Model Accuracy Speed (samples/s) Time (s)  Correct  Total
LogisticRegression   0.9680             922.9   75.848    67757  70000
               SVM   0.8810             640.9  109.224    61670  70000
               MLP   0.9371             865.4   80.887    65595  70000
  GradientBoosting   0.9471             882.5   79.323    66297  70000

En yüksek accuracy: LogisticRegression (0.9680)
En hız